In [1]:
# Esto es para quienes tengan jupiter corriendo en vscode.

# %pip install -U pip setuptools wheel
# %pip install -U spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm

In [2]:
import numpy as np
import pandas as pd

En lo que sigue, tener bien en claro, que tanto ***CountVectorizer*** como ***TfidfVectorizer*** son clases ***"transformadores"***, es decir, se aplican a ***"X"***, (variables preditivas).  No son estimadores. Son parecidos a Scaler, scaler también es un transformador.  
  
El objetivo tanto de ***CountVectorizer*** como de ***TfidfVectorizer*** es que, a partir de las ***"X"*** ***"textuales"*** obtener ***"X"*** ***"numéricas"*** ,en donde, cada número o elemento de esta nueva matriz (la de salida) represente, según ciertos criterios, la importancia de cada elemento textual de la ***X original*** en cada registro.   
Como resultado final se obtiene una matriz en donde las filas representan los registros y las columnas son cada uno de los elementos textuales únicos de la matriz ***"X"*** original. 
  
     
Para entenderlo mejor, se expone un ejemplo simple, con un dataframe original con 3 registros textuales.   
En este caso no se incluye la variable objetivo ***"y"*** solo por claridad. Es decir en este caso df=X.  
  
Nota:   
Notar que tanto CountVectorizer como TfidfVectorizer no procesan Stop_words como "y", "el", etc.

In [3]:
# DataFrame de ejemplo X original.
data = {
    'Registros': [
        'Este es un ejemplo de texto de prueba',
        'Otro ejemplo similar al ejemplo anterior',
        'Y Otro ejemplo más corto'
    ]
}
X = pd.DataFrame(data)
display(X)

,Registros
0,Este es un ejemplo de texto de prueba
1,Otro ejemplo similar al ejemplo anterior
2,Y Otro ejemplo más corto


#### CountVectorizer
Convierte los "Registros" de "texto" en registros de ***recuentos*** de ***términos*** (elementos numéricos).   
Primero Crea el ***"corpus"*** un ***conjunto con todas las palabras o ngramas u objetos únicos*** de todos los documentos o registros.    
   
Y luego cuenta la cantidad de veces que aparece cada elemento del corpus en cada registro.  
    
El resultado final es una matriz cuyas columnas son los elementos únicos del corpus y sus filas son los registros, de modo que, cada elemento de la matriz indica, la cantidad de veces que aparece ese elemento (el del corpus) en el registro original.  

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Crear una instancia de CountVectorizer
vectorizer = CountVectorizer()

# Ajustar y transformar los datos de texto en una matriz numérica
matriz_numerica = vectorizer.fit_transform(X["Registros"])

# Convertir la matriz numérica a un DataFrame de Pandas
X_numerico = pd.DataFrame(matriz_numerica.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar el DataFrame numérico resultante
display(X)
display(X_numerico)

,Registros
0,Este es un ejemplo de texto de prueba
1,Otro ejemplo similar al ejemplo anterior
2,Y Otro ejemplo más corto


,al,anterior,corto,de,ejemplo,es,este,más,otro,prueba,similar,texto,un
0,0,0,0,2,1,1,1,0,0,1,0,1,1
1,1,1,0,0,2,0,0,0,1,0,1,0,0
2,0,0,1,0,1,0,0,1,1,0,0,0,0


#### TfidfVectorizer
Hace lo mismo, solo que los elemntos de la matriz devuelta, en vez de ser recuentos, son números calculados según:  
  
Si el parámetro smooth_idf=True (valor por defecto)
$$\text{TF-IDF}=N_t \cdot \Bigg(ln\Bigg[\frac{1+ND}{1+d(t)}\Bigg]+1\Bigg)$$
Ó si el parámetro smooth_idf=False
$$\text{TF-IDF}=N_t \cdot \Bigg(ln\Bigg[\frac{ND}{d(t)}\Bigg]+1\Bigg)$$

y finalmente "Normalizados" según:
$$\text{TF-IDF}_{norm}=\frac{\text{TF-IDF}}{\sqrt{\displaystyle\sum_{i=1}^{nd}(\text{TF-IDF}_i)^2}}$$
en donde:  
  
$N_t$ es el número de veces que aparece la palabra o elemento "$t$" en el registro o documento en cuestión.  
$ND$ es el número total de registros o documentos.  
$d(t)$ es el número de registros o documentos en los que aparece el elemento "$t$"  
$nd$ es el número de términos o elementos únicos en el registro o documento "$d$".

#### Ejemplo manual, para el primer registro  
'Registros': [  
        'Este es un ejemplo de texto de prueba',  
        'Otro ejemplo similar al ejemplo anterior',  
        'Y Otro ejemplo más corto'  
    ]  
  
$N_t$ = "este" aparece 1 vez en el primer registro.  
$ND$ = hay 3 registros.  
d("este")= "este" sólo aparece en el primer registro. = 1  
nd = cantidad de elementos ***únicos*** en el primer registro = 7    
  
  $$\text{TF-IDF}("este")=1 \cdot \Bigg(ln\Bigg[\frac{1+3}{1+1}\Bigg]+1\Bigg)=1.69314718$$

Lo mismo para "es" , "un" , "texto" , y "prueba"  
Luego para "ejemplo":  
$$\text{TF-IDF}("ejemplo")=1 \cdot \Bigg(ln\Bigg[\frac{1+3}{1+3}\Bigg]+1\Bigg)=1$$
y para "de":  
$$\text{TF-IDF}("de")=2 \cdot \Bigg(ln\Bigg[\frac{1+3}{1+1}\Bigg]+1\Bigg)=3.38629436$$
y la norma sería:  
$$\sqrt{\displaystyle\sum_{i=1}^{nd}(\text{TF-IDF}_i)^2}=\sqrt{5\cdot 1.69314718^2+1^2+3.38629436^2}=5.17694180$$
    
y por lo tanto los valores TDIDF ***Normalizados*** serán:  
$$\text{TF-IDF}("este")=\text{TF-IDF}("es")=\text{TF-IDF}("un")=\text{TF-IDF}("texto")=\text{TF-IDF}("prueba")=\frac{1.69314718}{5.17694180}=0.327055$$
$$\text{TF-IDF}("ejemplo")=\frac{1}{5.17694180}=0.19316423$$
$$\text{TF-IDF}("de")=\frac{3.38629436}{5.17694180}=0.65411096$$

In [28]:
import math
este=1*(math.log(2)+1)
ejemplo=1*(math.log(1)+1)
de=2*(math.log(2)+1)
norma=np.sqrt(5*este**2+ejemplo**2+de**2)

norma,este/norma,ejemplo/norma,de/norma

(5.176941797561842,
 0.32705547923242223,
 0.19316423462032448,
 0.6541109584648445)

Se verifica lo anterior con los resultados que devuelve ***TfidfVectorizer***

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar el TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Aplicar el TfidfVectorizer a la columna 'texto'
tfidf_matrix = tfidf_vectorizer.fit_transform(X['Registros'])

# Crear un DataFrame a partir de la matriz TF-IDF
X_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=X.index)

# Mostrar el DataFrame resultante
display(X)
display(X_tfidf)

,Registros
0,Este es un ejemplo de texto de prueba
1,Otro ejemplo similar al ejemplo anterior
2,Y Otro ejemplo más corto


,al,anterior,corto,de,ejemplo,es,este,más,otro,prueba,similar,texto,un
0,0.000000,0.000000,0.000000,0.654111,0.193164,0.327055,0.327055,0.000000,0.000000,0.327055,0.000000,0.327055,0.327055
1,0.448394,0.448394,0.000000,0.000000,0.529657,0.000000,0.000000,0.000000,0.341015,0.000000,0.448394,0.000000,0.000000
2,0.000000,0.000000,0.584483,0.000000,0.345205,0.000000,0.000000,0.584483,0.444514,0.000000,0.000000,0.000000,0.000000
